### RETRIEVAL AUGMENTED GENERATION

<img src="../assets/rag_macro.png" alt="drawing" width="600"/>

#### Initialisation du model
Nous réutilisons le même modèle que pour le chat bot simple

In [7]:
from dotenv import load_dotenv
from os import getenv
from langchain_openai import ChatOpenAI

load_dotenv()


model = ChatOpenAI(model="gpt-4o")

#### Preparation du Prompt Template

Nous procédons à deux évolutions dans le `prompt template`:
* Le message System évolue pour préciser ce qui est attendu du modèle
* Un nouveau placeholder est créé pour y injecter les documents pertinents à fournir au modèle pour générer la réponse


In [ ]:
from langchain_core.prompts import ChatPromptTemplate 

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an assistant for question-answering tasks. 
         Use the following pieces of retrieved context to answer the question. 
         If you don't know the answer, just say that you don't know. 
         Use three sentences maximum and keep the answer concise."""),
        ("placeholder", "{history}"),
        ("system", "{context}"),
        ("human", "{question}")
    ]
)

print(prompt_template)

#### Initialisation de la mémoire
Pour la mémoire nous utiliserons cette fois-ci une instance Redis locale

In [9]:
from langchain_community.chat_message_histories import RedisChatMessageHistory

history = RedisChatMessageHistory("genai_in_action_simple_rag", url=getenv("REDIS_URL"))

#### Ingestion du document

<img src="../assets/rag_load_split_embed.png" alt="drawing" width="900"/>

Pour l'étape `Load`, nous utilisons un des nombreux `Loader` natif de Langchain, ici celui traitant les fichiers `txt`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# LOAD
loader = PyPDFLoader("../__docs__/NO-GRH-010 Gestion des déplacements et défraiement_1.0.pdf")
docs = loader.load()
print(docs[0].page_content[:250])
print(docs[0].metadata)

Pour le `Split`, le document est découpé en *chunk* de 1000 caractères avec un recoupement (*overlap*) entre eux de 200 caractères.

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

# SPLIT
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

splits

Enfin pour la partie `Embed`et `Store`, nous faisons appel au modèle de LLM d'embedding d'penAi (`ada`) pour réaliser les embeddings et les stockons dans le vector database du [Redis local](http://localhost:8001/redis-stack/browser)

In [12]:
from langchain_community.vectorstores.redis import Redis
from langchain_openai import OpenAIEmbeddings


vectorstore = Redis.from_documents(
    splits,
    OpenAIEmbeddings(),
    redis_url=getenv("REDIS_URL"),
    index_name="genai_in_action_simple_rag",
)

#### Création de la chaîne OpenAI

La chaîne finale se complexifie pour être une composition de plusieurs chaînes:

La chaîne principale est une chaîne séquentielle qui va :
1. Traiter les inputs
2. Passer ces inputs traités au prompt template
3. Passer le prompt termplate variabilisé au model
4. Passer la réponse à un parser pour en extraire simplement la réponse au format texte

Le premier élément de la chaîne est elle-même une chaîne qui va traiter les inputs en parrallèle:
* `question` et `history` sont simplement passé tels quels
* `context` est produit via une 3ème chaîne séquentielle qui va à partir de la question de l'utilistaeur retourné les bouts de document les plus pertinents du vector store

In [13]:
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    # Runnable parallèles
    {
        "context": itemgetter("question") | vectorstore.as_retriever() | format_docs, 
        "question": itemgetter("question"), 
        "history": itemgetter("history")
    }
    # Runnable séquentiels
    | prompt_template
    | model
    | StrOutputParser()
)


On peut maintenant poser des questions sur le document

In [ ]:
question1 ="Quelle est la politique de déplacement de Younup ?"

response1 = rag_chain.invoke(
    {
        "question": question1,
        "history": history.messages
    }
)
response1

#### Bonus: Streamer la réponse

In [ ]:
question2 ="Quelles sont les personnes à contacter ?"
response2 = ""

for chunk in rag_chain.stream(
    {
        "question": question2,
        "history": history.messages
    }):
    print(chunk)
    response2+=chunk

print(f"Réponse complète: {response2}")

